In [25]:
import os, json, math, time
# Standard Imports
import pandas as pd
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'superhero_info.csv',
 'superhero_powers - superhero_powers.csv',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_title.csv.gz',
 'tmbd_api_results2000.json',
 'tmbd_api_results2001.json',
 'tmdb_api_results_.json']

In [2]:
import json
with open("/Users/Meryum Syeda/.secret/tmdb_api.json", "r") as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [4]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [5]:
## movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/n2nm4aZRmXyJ9LT4xQX9X6ThcP7.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 76.758,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

In [6]:
info['revenue']

463517383

In [7]:
info['imdb_id']

'tt0133093'

In [8]:
info['budget']

63000000

In [9]:
YEARS_TO_GET = [2000,2001]

In [10]:
# Function to get movie ratings
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
             movie_info['certification'] = c['certification']
    return movie_info

In [20]:
def write_json(new_data, filename): 
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)
    
    # If it does not exist: create it
        if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
           with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)

In [21]:
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    
    JSON_FILE = f'{FOLDER}tmbd_api_results{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: create it
    if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
    
    # Filter the IMDB title_basics file on current Year
    df = basics.loc[ basics['startYear']==YEAR].copy()
    
    # Save IMDB title_basics file movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

#Get index and movie id from list


# INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
    
# Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue
            
   #Saving new year as the current df
    #df = basics.loc[ basics['startYear']==YEAR].copy()
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1508 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/265 [00:00<?, ?it/s]

In [57]:
df_1 = pd.read_csv("Data/final_tmdb_data_2000.csv.gz")
df_2 = pd.read_csv("Data/final_tmdb_data_2001.csv.gz")

In [58]:
df = df_1.append(df_2)

C:\Users\Meryum Syeda\AppData\Local\Temp\ipykernel_19176\2979329560.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_1.append(df_2)


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2618 entries, 0 to 1343
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2618 non-null   object 
 1   adult                  2616 non-null   float64
 2   backdrop_path          1327 non-null   object 
 3   belongs_to_collection  197 non-null    object 
 4   budget                 2616 non-null   float64
 5   genres                 2616 non-null   object 
 6   homepage               188 non-null    object 
 7   id                     2616 non-null   float64
 8   original_language      2616 non-null   object 
 9   original_title         2616 non-null   object 
 10  overview               2562 non-null   object 
 11  popularity             2616 non-null   float64
 12  poster_path            2290 non-null   object 
 13  production_companies   2616 non-null   object 
 14  production_countries   2616 non-null   object 
 15  rele

In [60]:
budget_filter = df['budget']>0
budget_filter.head()
df = df.loc[budget_filter, :]
df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.106,1957.0,PG
6,tt0119273,0.0,/fClJrAmJQ90zg9gWnfHVYSMiDTv.jpg,"{'id': 141086, 'name': 'Heavy Metal Collection...",15000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,16225.0,en,Heavy Metal 2000,...,0.0,88.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,not to survive the fight in the external world...,Heavy Metal 2000,0.0,6.086,157.0,R
11,tt0120467,0.0,/knok3mNReKqPTplnnqz7E4dd7mD.jpg,NaN,120000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",NaN,19085.0,en,Vulgar,...,14904.0,87.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Everyone loves a clown... some more than others.,Vulgar,0.0,5.400,42.0,R
12,tt0120626,0.0,/6xz9H7IMK3W21SNwAVmE9oW6YjX.jpg,NaN,6105121.0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",NaN,54380.0,pt,Capitães de Abril,...,0.0,123.0,"[{'english_name': 'Portuguese', 'iso_639_1': '...",Released,NaN,Captains of April,0.0,6.400,47.0,NaN


In [61]:
revenue_filter = df['revenue']>0
revenue_filter.head()
df = df.loc[revenue_filter, :]
df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.106,1957.0,PG
11,tt0120467,0.0,/knok3mNReKqPTplnnqz7E4dd7mD.jpg,NaN,120000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",NaN,19085.0,en,Vulgar,...,14904.0,87.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Everyone loves a clown... some more than others.,Vulgar,0.0,5.400,42.0,R
13,tt0120630,0.0,/sPAwM8WxMdXNlqeP4F9DMe9LYoY.jpg,"{'id': 718551, 'name': 'Chicken Run Collection...",45000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,7443.0,en,Chicken Run,...,224834564.0,84.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,This ain't no chick flick. It's poultry in mot...,Chicken Run,0.0,6.726,4109.0,G
16,tt0120753,0.0,NaN,NaN,8000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,318.0,en,The Million Dollar Hotel,...,105983.0,122.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Everyone has something to hide.,The Million Dollar Hotel,0.0,5.915,254.0,R
17,tt0120755,0.0,/24DZfupDlhXeTchmcOkoGRhP5Vg.jpg,"{'id': 87359, 'name': 'Mission: Impossible Col...",125000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",NaN,955.0,en,Mission: Impossible II,...,546388105.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Expect the impossible again.,Mission: Impossible II,0.0,6.109,5448.0,PG-13


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 4 to 1299
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                351 non-null    object 
 1   adult                  351 non-null    float64
 2   backdrop_path          338 non-null    object 
 3   belongs_to_collection  84 non-null     object 
 4   budget                 351 non-null    float64
 5   genres                 351 non-null    object 
 6   homepage               57 non-null     object 
 7   id                     351 non-null    float64
 8   original_language      351 non-null    object 
 9   original_title         351 non-null    object 
 10  overview               351 non-null    object 
 11  popularity             351 non-null    float64
 12  poster_path            350 non-null    object 
 13  production_companies   351 non-null    object 
 14  production_countries   351 non-null    object 
 15  relea

Below we can see there are 140 R-rated movies included, 117 PG-13, 29 PG, 13 G-rated, and 6 that do not have a rating.

In [63]:
df['certification'].value_counts()

R        140
PG-13    117
PG        29
G         13
NR         6
Name: certification, dtype: int64

In [64]:
df["certification"]

4          PG
11          R
13          G
16          R
17      PG-13
        ...  
1027      NaN
1059      NaN
1167      NaN
1280      NaN
1299      NaN
Name: certification, Length: 351, dtype: object

In [65]:
#Average revenue per certification category
df.groupby('certification')['revenue'].mean()

certification
G        1.332169e+08
NR       2.382332e+07
PG       1.346138e+08
PG-13    1.107121e+08
R        5.344654e+07
Name: revenue, dtype: float64

In [66]:
#Average budeget per certification category
df.groupby('certification')['budget'].mean()

certification
G        4.400000e+07
NR       1.478333e+07
PG       5.374138e+07
PG-13    4.680682e+07
R        2.750255e+07
Name: budget, dtype: float64